In [ ]:
!unzip /content/dataset_anemia_preprocessed.zip

In [ ]:
#problems:,preprocessing only 116 italy out of 123 images
#NOtes:augmentaion failed on jupyter,using albumnetations,some transparent some white bsckgroung, some no alpha
#augentation parameters

Add mask logic,finalize augmentation

#Augmentation to Only Training Data

In [ ]:
!unzip "/content/drive/MyDrive/BTP 1/dataset anemia.zip"

In [ ]:
import shutil
import os

shutil.rmtree("/content/augmented_dataset", ignore_errors=True)
print("Deleted folder: anemia_augmented")

csv_path=OUTPUT_CSV


if os.path.exists(csv_path):
    os.remove(csv_path)
    print("Deleted:", csv_path)

# if os.path.exists(zip_path):
#     os.remove(zip_path)
#     print("Deleted:", zip_path)

Deleted folder: anemia_augmented


NameError: name 'OUTPUT_CSV' is not defined

##Helper Functions

In [ ]:
# =======================================================
# 🧩 Morphological Opening Function  (YOUR EXACT LOGIC)
# =======================================================
def apply_morph_opening(image_path):
    img = load_corrected_image(image_path)
    if img is None:
        return None

    if img.dtype == np.uint16:
        img = (img / 256).astype(np.uint8)

    # Work in BGR directly
    bgr = img.copy()

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    opened_bgr = np.zeros_like(bgr)

    for c in range(3):
        opened_bgr[:, :, c] = cv2.morphologyEx(bgr[:, :, c], cv2.MORPH_OPEN, kernel, iterations=2)

    return opened_bgr   # <--- NOW RETURN BGR



def apply_aug_whole_image(image,aug):
    # ---- Run aug with mask together (IMPORTANT) ----
    aug_out = aug(image=image)

    aug_img = aug_out["image"]
    # new_mask = aug_out["mask"]

    # White-out outside conjunctiva region
    # aug_img[new_mask == 0] = 255

    return aug_img


from PIL import Image, ImageOps, UnidentifiedImageError

def load_corrected_image(path):
    try:
        img = Image.open(path)
        img = ImageOps.exif_transpose(img)  # auto-fix EXIF rotation
        img = np.array(img)                # -> numpy RGB
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        return img
    except (UnidentifiedImageError, OSError, ValueError):
        print(f"[CORRUPT] Skipping file: {path}")
        return None



##MAML Setup

In [ ]:
import albumentations as A

# --------------------------------------------------
# 1. SINGLE DICTIONARY OF ALL ATOMIC AUGMENTATIONS
# --------------------------------------------------
AUG = {
    "affine": A.Affine(
        scale=(0.85, 1.15),
        rotate=(-12, 12),
        shear=(-4, 4),
        p=1.0
    ),

    "color": A.ColorJitter(
        brightness=0.20,
        contrast=0.20,
        saturation=0.12,
        hue=0.02,
        p=1.0
    ),

    "gauss_noise": A.GaussNoise(
        std_range=(0.15, 0.25),
        p=1.0
    ),

    "iso_noise": A.ISONoise(
        intensity=(0.25, 0.50),
        color_shift=(0.02, 0.07),
        p=1.0
    ),

    "motion_blur": A.MotionBlur(
        blur_limit=(5, 15),
        p=1.0
    ),

    "gaussian_blur": A.GaussianBlur(
        blur_limit=(5, 9),
        p=1.0
    ),

    "downscale": A.Downscale(
        scale_range=(0.60, 0.85),
        p=1.0
    ),

    "compression": A.ImageCompression(
        quality_range=(60, 85),
        p=1.0
    ),
}
PIPE = {
    # -----------------------
    # 1–7: Single Augmentations
    # -----------------------
    "Aug1":  ["affine"],
    "Aug2":  ["color"],
    "Aug3":  ["gauss_noise"],
    "Aug4":  ["iso_noise"],
    "Aug5":  ["motion_blur"],
    "Aug6":  ["gaussian_blur"],
    "Aug7":  ["downscale","compression"],

    # -----------------------
    # 8–13: Combined Augmentations
    # -----------------------
    "Aug8":   ["color", "motion_blur"],
    "Aug9":   ["affine", "color"],
    "Aug10":  ["affine", "gaussian_blur"],
    "Aug11":  ["color", "gauss_noise"],
    "Aug12":  ["gaussian_blur", "gauss_noise"],
    "Aug13":  ["iso_noise", "compression"],
}

def get_pipeline(name):
    return A.Compose([AUG[a] for a in PIPE[name]])

def MAML_process_split(df_split, mode="train"):
    """
    df_split : DataFrame (train or valid)
    mode : "train" or "valid"
    """
    assert mode in ["train", "valid"]

    # Output folder
    OUT_DIR = os.path.join(OUTPUT_IMG_DIR, mode)
    os.makedirs(OUT_DIR, exist_ok=True)

    records = []

    for _, row in tqdm(df_split.iterrows(), total=len(df_split), desc=f"Processing {mode}"):
        region = row["Region"]
        pid = str(int(row["Number"]))
        age = row["Age"]
        gender = row["Gender"]
        hgb = row["Hgb"]

        # Patient directory
        patient_dir = os.path.join(ROOT_DIR, region, pid)

        # Find conjunctiva image
        orig_file = None
        for f in os.listdir(patient_dir):
            if "forniceal_palpebral" in f.lower() and f.lower().endswith((".png", ".jpg", ".jpeg")):
                orig_file = f
                break

        if orig_file is None:
            print(f"[WARNING] No image found in {patient_dir}")
            continue

        img_path = os.path.join(patient_dir, orig_file)

        # Morph preprocessing
        processed, mask = apply_morph_opening(img_path)
        if processed is None:
            continue

        # Create output directory
        out_dir = os.path.join(OUT_DIR, region, pid)
        os.makedirs(out_dir, exist_ok=True)

        # Save processed original
        orig_save_path = os.path.join(out_dir, "original.png")
        cv2.imwrite(orig_save_path, processed)

        # Add original to CSV
        records.append([orig_save_path, age, gender, hgb, region])

        # -------------------------
        # 🌟 TRAIN AUGMENTATION ONLY
        # -------------------------
        if mode == "train":
            img_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)

            for k in range(N_AUGS):
                aug = get_pipeline(f"Aug{k+1}")
                aug_img = apply_aug_inside_mask(img_rgb, aug, mask)
                aug_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

                aug_name = f"aug_{k}.png"
                aug_path = os.path.join(out_dir, aug_name)

                cv2.imwrite(aug_path, aug_bgr)

                records.append([aug_path, age, gender, hgb, region])

    # -----------------------
    # Create final DataFrame
    # -----------------------
    out_df = pd.DataFrame(records, columns=["image_path", "Age", "Gender", "Hgb", "Region"])

    # Encoding
    out_df["Gender"] = out_df["Gender"].map({"M": 1, "F": 0}).astype(float)
    out_df["Region"] = out_df["Region"].map({"India": 0, "Italy": 1}).astype(float)

    # Clean Hgb
    out_df["Hgb"] = pd.to_numeric(out_df["Hgb"], errors="coerce")
    out_df = out_df.dropna(subset=["Hgb"])

    # Normalize Age (Age/100)
    out_df["Age"] = out_df["Age"] / 100.0

    return out_df


##BPA_Net Setup

In [ ]:
import albumentations as A

# ----------------------------------------
# BEST 5 AUGMENTATIONS WITH PROBABILITIES
# ----------------------------------------
AUG_PIPELINE = A.Compose([
    A.Affine(
        scale=(0.85, 1.15),
        rotate=(-12, 12),
        shear=(-4, 4),
        p=0.35
    ),

    A.ColorJitter(
        brightness=0.20,
        contrast=0.20,
        saturation=0.12,
        hue=0.02,
        p=0.25
    ),

    A.GaussNoise(
        std_range=(0.15, 0.25),
        p=0.30
    ),

    A.GaussianBlur(
        blur_limit=(5, 9),
        p=0.25
    ),

    A.ImageCompression(
            quality_range=(60, 85),
            p=0.40
)
])

def BPANet_process_split(df_split, mode="train"):
    """
    df_split : DataFrame (train or valid)
    mode : "train" or "valid"
    """
    assert mode in ["train", "valid"]

    OUT_DIR = os.path.join(OUTPUT_IMG_DIR, mode)
    os.makedirs(OUT_DIR, exist_ok=True)

    records = []

    for _, row in tqdm(df_split.iterrows(), total=len(df_split), desc=f"Processing {mode}"):
        region = row["Region"]
        pid = str(int(row["Number"]))
        age = row["Age"]
        gender = row["Gender"]
        hgb = row["Hgb"]

        patient_dir = os.path.join(ROOT_DIR, region, pid)

        orig_file = None
        for f in os.listdir(patient_dir):
            if ("forniceal" not in f.lower()) and ("palpebral" not in f.lower()) and f.lower().endswith((".png", ".jpg", ".jpeg")):
                orig_file = f
                break

        if orig_file is None:
            print(f"[WARNING] No image found in {patient_dir}")
            continue

        img_path = os.path.join(patient_dir, orig_file)

        # Only morph opening now (mask removed)
        processed = apply_morph_opening(img_path)
        if processed is None:
            continue

        out_dir = os.path.join(OUT_DIR, region, pid)
        os.makedirs(out_dir, exist_ok=True)

        # ------------------------------
        # VALID → save original only
        # ------------------------------
        if mode == "valid":
            save_path = os.path.join(out_dir, "original.png")
            cv2.imwrite(save_path, processed)

            records.append([save_path, age, gender, hgb, region])

        else:  # ------------------- TRAIN AUG -------------------
            img_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)

            aug_img = apply_aug_whole_image(img_rgb, AUG_PIPELINE)
            aug_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

            aug_path = os.path.join(out_dir, f"aug_{pid}.png")
            cv2.imwrite(aug_path, aug_bgr)

            records.append([aug_path, age, gender, hgb, region])

    # --------------------
    # Build output dataframe
    # --------------------
    out_df = pd.DataFrame(records, columns=["image_path", "Age", "Gender", "Hgb", "Region"])

    out_df["Gender"] = out_df["Gender"].map({"M": 1, "F": 0}).astype(float)
    out_df["Region"] = out_df["Region"].map({"India": 0, "Italy": 1}).astype(float)

    out_df["Hgb"] = pd.to_numeric(out_df["Hgb"], errors="coerce")
    out_df = out_df.dropna(subset=["Hgb"])

    out_df["Age"] = out_df["Age"] / 100.0

    return out_df

##Augmentation based on setup

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import albumentations as A
from tqdm import tqdm
import albumentations as A
from sklearn.preprocessing import MinMaxScaler
# Split BEFORE augmentation
from sklearn.model_selection import train_test_split

# =======================================================
# 🔧 CONFIG
# =======================================================
ROOT_DIR = "/content/dataset anemia"
INDIA_XLSX = os.path.join(ROOT_DIR, "India", "India.xlsx")
ITALY_XLSX = os.path.join(ROOT_DIR, "Italy", "Italy.xlsx")

OUTPUT_IMG_DIR = "/content/augmented_dataset"
os.makedirs(OUTPUT_IMG_DIR, exist_ok=True)

N_AUGS = 13

# =======================================================
# LOAD METADATA
# =======================================================
df_india = pd.read_excel(INDIA_XLSX)
df_india["Region"] = "India"

df_italy = pd.read_excel(ITALY_XLSX)
df_italy["Region"] = "Italy"

df = pd.concat([df_india, df_italy], ignore_index=True)

df_train, df_val = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=df["Region"]
)

train_df = BPANet_process_split(df_train, mode="valid")
train_df.to_csv("train.csv", index=False)

valid_df = BPANet_process_split(df_val, mode="valid")
valid_df.to_csv("valid.csv", index=False)


Processing valid:  78%|███████▊  | 136/174 [02:10<00:33,  1.15it/s]

[CORRUPT] Skipping file: /content/dataset anemia/Italy/95/T_64_20190612_092742_palplebral.png


Processing valid: 100%|██████████| 44/44 [00:41<00:00,  1.07it/s]


#Download Augmented Data & CSV

In [ ]:
import os
import shutil
from google.colab import files

def download_outputs(folder_path, csv_path_1, csv_path_2):

    # 1. ZIP + DOWNLOAD FOLDER
    if os.path.exists(folder_path):
        zip_name = "augmented_dataset"
        shutil.make_archive(zip_name, "zip", folder_path)
        print(f"Zipped folder: {zip_name}.zip")
        files.download(f"{zip_name}.zip")
    else:
        print(f"❌ Folder not found: {folder_path}")

    # 2. DOWNLOAD CSV 1
    if os.path.exists(csv_path_1):
        print(f"CSV found → downloading: {csv_path_1}")
        files.download(csv_path_1)
    else:
        print(f"⚠️ CSV not found at: {csv_path_1}")

    # 3. DOWNLOAD CSV 2
    if os.path.exists(csv_path_2):
        print(f"CSV found → downloading: {csv_path_2}")
        files.download(csv_path_2)
    else:
        print(f"⚠️ CSV not found at: {csv_path_2}")


# ✅ Correct call
download_outputs(
    folder_path="/content/augmented_dataset",
    csv_path_1="/content/train.csv",
    csv_path_2="/content/valid.csv"
)


Zipped folder: augmented_dataset.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV found → downloading: /content/train.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV found → downloading: /content/valid.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>